# Scrape FIR information

With this notebook we retrieve the informations about FIRs (Flight Information Region) from wikipedia.

In [1]:
from bs4 import BeautifulSoup
import numpy as np
from datetime import datetime
import time
import urllib.request

Request the html code of the page and parse it with BeutifoulSoup (HTML parser)

In [2]:
fp = urllib.request.urlopen("https://en.wikipedia.org/wiki/Flight_information_region")
mybytes = fp.read()

html = mybytes.decode("utf8")
fp.close()
soup = BeautifulSoup(html)

Find the information with soup and store them in a matrix.

### IMPORTANT
In wikipedia there are 5 columns (ICAO, Type, ACC name, Oceania Subtype, Country/Territory) but we are
interested only in 4 of these, that are ICAO, Type, ACC name and country.

In [3]:
table = soup.find("table", recursive=True)
caption = table.find("caption",recursive =True)

header = ["ICAO","Type","ACC_name","country"]
'''
Order of tds
ICAO
Type
ACC name (Area control centre)
Oceanica subtype
Country/Territory
'''

tbody = table.find("tbody",recursive=True)
trs = tbody.find_all("tr",recursive=True)
i=0
rows = 0
#data = np.empty((1,6))
print(len(trs))
data = np.empty(0)
for tr in trs:
    #first tr is the header
    if i==0:
        i+=1
    else:
        tds = tr.find_all("td",recursive=True)
        line = []
        j=0
        for td in tds:
            valid = True
            if j == len(tds)-1:
                text = td.find("a",recursive = True).getText().strip()
                if text.startswith("Congo"):
                    text = "Congo"
            elif j!=3:
                text = td.getText().replace("\n","").replace(",",";")
                
            else:
                valid = False
            if valid:
                line.append(text)
            j+=1
        data = np.append(data,line)
        rows+=1
        data = data.reshape(rows,4)

data = data.reshape(rows,4)
print(data.shape)

309
(308, 4)


In [4]:
#save to csv
head = ','.join(header)
print(data.shape)
np.savetxt('fir_wiki.csv', data, header=head, fmt=['%s','%s','%s','%s'],delimiter=',')

(308, 4)
